In [1]:
import torch

## 张量
在PyTorch中， torch.Tensor 是存储和变换数据的主要⼯工具。如果你之前⽤用过NumPy，你会发现 Tensor 和 NumPy 的多维数组⾮非常类似。然⽽，Tensor 提供GPU计算和⾃自动求梯度等更更多功能，这些使 Tensor 更加适合深度学习。  

"tensor"这个单词一般可译作“张量”，张量可以看作是⼀一个多维数组。标量可以看作是0维张量，向量可以看作1维张量，矩阵可以看作是⼆维张量。

In [2]:
x = torch.empty(5,3)
print(x)

tensor([[1.0561e-38, 1.0929e-38, 1.0102e-38],
        [4.5918e-39, 1.0561e-38, 1.0561e-38],
        [1.0561e-38, 1.0745e-38, 1.0561e-38],
        [8.7245e-39, 9.6429e-39, 9.6429e-39],
        [8.7245e-39, 4.2246e-39, 1.1112e-38]])


In [3]:
torch.rand(5, 3)

tensor([[0.8549, 0.2935, 0.7408],
        [0.4818, 0.3327, 0.1787],
        [0.4807, 0.5494, 0.5236],
        [0.3543, 0.7973, 0.6606],
        [0.1006, 0.4827, 0.1958]])

In [5]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [3]:
x = torch.tensor([5.5, 3])
print(x)

tensor([5.5000, 3.0000])


In [4]:
x = x.new_ones(5, 3, dtype = torch.float64)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


In [6]:
print(x.size())
print(x.shape)

torch.Size([5, 3])
torch.Size([5, 3])


## 索引
我们还可以使⽤用类似NumPy的索引操作来访问 Tensor 的⼀一部分，需要注意的是：**索引出来的结果与原数据共享内存，也即修改一个，另一个会跟着修改**。

In [8]:
y = x[0, :]
print(y)
y += 1
print(y)
print(x)     # 源tensor也被改变

tensor([1., 1., 1.], dtype=torch.float64)
tensor([2., 2., 2.], dtype=torch.float64)
tensor([[2., 2., 2.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


### 改变形状

In [10]:
y = x.view(15)
print(y)
z = x.view(-1, 5)   # -1所指的维度可以根据其他维度的值推出来
print(z)

tensor([2., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64)
tensor([[2., 2., 2., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]], dtype=torch.float64)


注意 view() 返回的新tensor与源tensor共享内存（其实是同一个tensor），也即更改其中的一个，另外⼀个也会跟着改变。(顾名思义，view仅仅是改变了了对这个张量的观察角度)

In [12]:
x -= 1
print(x)
print(y)
print(z)    # 全部都会改变

tensor([[ 0.,  0.,  0.],
        [-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.]], dtype=torch.float64)
tensor([ 0.,  0.,  0., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1.], dtype=torch.float64)
tensor([[ 0.,  0.,  0., -1., -1.],
        [-1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1.]], dtype=torch.float64)


reshape()可以改变形状同时返回一个真正的副本（即不共享内存），但此函数并不能保证返回的是其拷贝，所以不推荐使用，推荐现有clone创造一个副本，再使用view。

In [13]:
x_cp = x.clone().view(15)
print(x_cp)

tensor([ 0.,  0.,  0., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1.], dtype=torch.float64)


In [14]:
x += 1
print(x)
print(x_cp)

tensor([[1., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)
tensor([ 0.,  0.,  0., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1.], dtype=torch.float64)


## 广播机制

In [15]:
x = torch.arange(1, 3).view(1, 2)
print(x)
y = torch.arange(1, 4).view(3, 1)
print(y)
print(x + y)

tensor([[1, 2]])
tensor([[1],
        [2],
        [3]])
tensor([[2, 3],
        [3, 4],
        [4, 5]])


## 内存开销
索引、 view 是不会开辟新内存的，而像 y = x + y 这样的运算是会新开内存的，然后将 y 指向新内存。为了演示这⼀一点，我们可以使用Python⾃自带的 id 函数：如果两个实例的ID⼀致，那么它们所对应的内存地址相同；反之则不同。

In [16]:
x = torch.tensor([1, 2])
y = torch.tensor([3, 4])
id_before = id(y)
y = y + x
print(id(y)==id_before)

False


如果想指定结果到原来的 y 的内存，我们可以使⽤用前⾯面介绍的索引来进行行替换操作。在下面的例子中，我们把 x + y 的结果通过 \[:\] 写进 y 对应的内存中。

In [17]:
x = torch.tensor([1, 2])
y = torch.tensor([3, 4])
id_before = id(y)
y[:] = y + x
print(id(y)==id_before)

True


我们还可以使⽤用运算符全名函数中的 out 参数或者自加运算符 += (也即 add_() )达到上述效果.

In [22]:
x = torch.tensor([1, 2])
y = torch.tensor([3, 4])
id_before = id(y)
# 以下方式都是可以的
#torch.add(x, y, out=y)
#y += x
y.add_(x)
print(id(y)==id_before)

True


## 梯度

In [3]:
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [4]:
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [5]:
print(x.is_leaf, y.is_leaf)

True False


In [6]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [16]:
a = torch.ones(2, 2)  # 缺省情况下 requires_grad = False
a += 2
a = (a * 2) / (a - 1)
print(a)
print(a.requires_grad)  # False
a.requires_grad_(True)
print(a.requires_grad)  # True
b = (a * a).sum()
print(a)
print(b)
print(b.grad_fn)

tensor([[3., 3.],
        [3., 3.]])
False
True
tensor([[3., 3.],
        [3., 3.]], requires_grad=True)
tensor(36., grad_fn=<SumBackward0>)


In [3]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

y = x + 2
print(y)

z = y * y * 3
out = z.mean()
print(z, out)

out.backward()
print(x.grad)   # out关于x的梯度

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [5]:
out2 = x.sum()
out2.backward()
print(x.grad)

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])


In [6]:
out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)

tensor([[1., 1.],
        [1., 1.]])


不允许张量对张量求导，只允许标量对张量求导，求导结果是和自变量同形的张量。

In [8]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


In [9]:
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype = torch.float)
z.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])
